# EDA

## Modules

In [2]:
# Set Global Variables
BASE_PATH='C:\\Users\\jonmc\\Documents\\git\\jonmccallum-okc-datascientist\\'

# Import Modules
import sys
import warnings

sys.path.insert(0, BASE_PATH + 'notebook\\resources\\')
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from process import *
import pandas_profiling as pf
import dtale
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis
import rpy2

%load_ext rpy2.ipython

# Analysis Configs

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## EDA Functions

In [3]:
# Functions
def plotHeatCorrMatrix(DF, path):
    corr_matrix = DF.corr(method='spearman')
    corr_list = (corr_matrix.abs()
                           .where(np.triu(np.ones(corr_matrix.shape), k=1)
                           .astype(bool))
                           .stack()
                           .sort_values(ascending=False))

    fig, ax = plt.subplots(figsize=(20,20))
    ax = sns.heatmap(corr_matrix
                     , mask=np.triu(np.ones_like(corr_matrix, dtype=bool))
                     , vmin=-1
                     , vmax=1
                     , center=0
                     , cmap = sns.diverging_palette(220,10, as_cmap=True)
                     , linewidths=0.5
                     , square=True
                     , annot=False
                     , fmt='g'
         )

    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

    plt.title('Correlation Among Numeric Features')
    plt.savefig(path)
    
    print(corr_list)
    
def makePandasProfileReport(DF, path):
    pfReportProfile = pf.ProfileReport(DF, title='Regression Challenge')
    pfReportProfile.to_file(BASE_PATH + path)

## Characterize Data

In [4]:
# Initial Exploration
print(getDir())

regressData = readCSV(BASE_PATH + 'data\\raw\\challenge1.csv')
regressData.dropna(how='any', inplace=True)
regressData.reset_index(inplace=True, drop=True)


print('\n', regressData.head(5),'\n')
print(regressData.tail(5),'\n')
regressData.dtypes

regressData = regressData.convert_dtypes()
print('\n', regressData.dtypes)

print(f'\nUnique values in "loc1": {regressData.loc1.unique()}')
print(f'\nUnique values in "loc2": {regressData.loc2.unique()}')

C:\Users\jonmc\Documents\git\jonmccallum-okc-datascientist\notebook\main\eda

   loc1 loc2  para1  dow  para2    para3  para4   price
0    0   01      1  Mon    662   3000.0    3.8   73.49
1    9   99      1  Thu    340   2760.0    9.2  300.00
2    0   04      0  Mon     16   2700.0    3.0  130.00
3    4   40      1  Mon     17  12320.0    6.4  365.00
4    5   50      1  Thu    610   2117.0   10.8  357.50 

     loc1 loc2  para1  dow  para2   para3  para4   price
9995    9   98      3  Fri    386  5000.0   12.0  460.00
9996    7   74      1  Thu    386  3250.0    8.0  325.00
9997    0   06      0  Tue    190  8856.0    5.6  133.33
9998    7   74      3  Fri    717  5000.0   13.6  820.00
9999    7   75      1  Thu    622   336.0    4.8  375.00 


 loc1      string
loc2      string
para1      Int64
dow       string
para2      Int64
para3    Float64
para4    Float64
price    Float64
dtype: object

Unique values in "loc1": <StringArray>
['0', '9', '4', '5', '7', '8', '1', '3', '2', '6', 'S

## Convert Factors to Dummies

In [5]:
# Make Dummy Columns and Plot Correlation Heatmap
factorCols = (regressData.drop(
                                regressData.columns[regressData.apply(lambda col: col.dtype) != 'string']
                                , axis=1
                         )
                         .columns
                         .tolist())

for col in factorCols:
    dummies = getDummies(regressData[col])
    regressData = mergeDF(regressData, dummies)
    
features = regressData.drop(columns=factorCols, axis=1)

## Create Heat Matrix of All Variables

In [6]:
plotHeatCorrMatrix(features, BASE_PATH + 'notebook\\main\\eda\\graphics\\correlation_matrix_all.png')

loc1_T   loc2_TS    1.000000
loc1_S   loc2_S6    1.000000
para3    para4      0.705182
para1    dow_Fri    0.641696
para4    price      0.634470
                      ...   
loc2_27  dow_Tue    0.000073
loc2_07  dow_Tue    0.000065
loc2_70  dow_Tue    0.000050
para1    loc2_93    0.000049
loc2_62  dow_Tue    0.000017
Length: 8515, dtype: float64


## Reduce Data - Multicollinearity

In [7]:
# Given correlation plot of natural grouping from loc2 to loc1, using only loc1 for parsimony sake
features = features.loc[:,~ features.columns.str.startswith('loc2_')]

# Dropping loc1_S, loc1_T, dow_Sat, and dow_Sun given the lack of variability
features = features.drop(columns=['loc1_S', 'loc1_T', 'dow_Sat', 'dow_Sun'], axis=1)
plotHeatCorrMatrix(features, BASE_PATH + 'notebook\\main\\eda\\graphics\\correlation_matrix_reduced.png')

para3   para4      0.705182
para1   dow_Fri    0.641696
para4   price      0.634470
para3   price      0.462563
para2   price      0.450712
                     ...   
loc1_1  dow_Fri    0.000649
loc1_3  dow_Tue    0.000613
para1   loc1_2     0.000573
loc1_9  dow_Thu    0.000349
loc1_5  dow_Wed    0.000076
Length: 190, dtype: float64


In [8]:
# Dropping dow_Fri and para3 given both are correlated with other regressors.
# However, the regressors they correlate with correlate less so with price than their respective complements
features = features.drop(columns=['dow_Fri', 'para3'], axis=1)

## Subset Data 
Generate:
- Target
- Regressors
- Combination of both

In [9]:
target = 'price'

combinedDF = features

# Removing target from regressors
regressors = features.drop(columns=[target], axis=1)

## Pandas / D-Tale Profiling 

In [10]:
makePandasProfileReport(combinedDF, path='notebook\\main\\eda\\graphics\\profile_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
dtaleUIReport = dtale.show(data=combinedDF, ignore_duplicates=False)
dtaleUIReport

## Sample Data

In [12]:
combinedDFSampled = combinedDF.sample(n=500, random_state=RANDOM_STATE)

In [13]:
writeCSV(combinedDFSampled, BASE_PATH + '\\data\\processed\\original_sample.csv')

## Profile Samples

In [14]:
makePandasProfileReport(combinedDFSampled, path='notebook\\main\\eda\\graphics\\sample_profile_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Scale

In [15]:
regressorsDFSampled = combinedDFSampled.drop(['price'], axis=1)
scaleRegressorsDF = scaleDataFrame(regressorsDFSampled)
scaleCombinedDF = scaleDataFrame(combinedDFSampled)


## K-Means

In [16]:
kPossible = [*range(1, len(scaleRegressorsDF.columns.tolist()) + 1)]
inertias = []

for k in kPossible:
    model = KMeans(n_clusters=k)
    
    model.fit(scaleRegressorsDF)
    
    inertias.append(model.inertia_)

fig, ax =plt.subplots(figsize=(20,20))
ax = plt.plot(kPossible, inertias, "-o")
plt.xlabel("Number of Clusters, k")
plt.ylabel("Inertia")
plt.xticks(kPossible)
plt.title("Scree of Explanatory Variables")
plt.savefig(BASE_PATH + 'notebook\\main\\eda\\graphics\\kmeans-scree.png')

In [17]:
# K-Means Analysis
kMeanDF = scaleCombinedDF.copy(deep=False)

kMeanModel = KMeans(n_clusters = 8)  # num clusters from scree
kMeanModel.fit(scaleCombinedDF)
kMeanDF['cluster_id'] = kMeanModel.predict(scaleCombinedDF)
kLabels = kMeanDF['cluster_id']

clusters_identified = np.unique(kLabels) + 1

In [18]:
kMeanDF = mergeDF(kMeanDF, combinedDF).rename(columns={'para1_x':'para1_scale'
                                                       , 'para2_x':'para2_scale'
                                                       , 'para4_x':'para4_scale'
                                                       , 'price_x':'price_scale'
                                                       , 'loc1_0_x':'loc1_0_scale'
                                                       , 'loc1_1_x':'loc1_1_scale'
                                                       , 'loc1_2_x':'loc1_2_scale'
                                                       , 'loc1_3_x':'loc1_3_scale'
                                                       , 'loc1_4_x':'loc1_4_scale'
                                                       , 'loc1_5_x':'loc1_5_scale'
                                                       , 'loc1_6_x':'loc1_6_scale'
                                                       , 'loc1_7_x':'loc1_7_scale'
                                                       , 'loc1_8_x':'loc1_8_scale'
                                                       , 'loc1_9_x':'loc1_9_scale'
                                                       , 'dow_Mon_x':'dow_Mon_scale'
                                                       , 'dow_Thu_x':'dow_Thu_scale'
                                                       , 'dow_Tue_x':'dow_Tue_scale'
                                                       , 'dow_Wed_x':'dow_Wed_scale'
                                                       , 'cluster_id':'cluster_id'
                                                       , 'para1':'para1'
                                                       , 'para2':'para2'
                                                       , 'para4':'para4'
                                                       , 'price':'price'
                                                       , 'loc1_0':'loc1_0'
                                                       , 'loc1_1':'loc1_1'
                                                       , 'loc1_2':'loc1_2'
                                                       , 'loc1_3':'loc1_3'
                                                       , 'loc1_4':'loc1_4'
                                                       , 'loc1_5':'loc1_5'
                                                       , 'loc1_6':'loc1_6'
                                                       , 'loc1_7':'loc1_7'
                                                       , 'loc1_8':'loc1_8'
                                                       , 'loc1_9':'loc1_9'
                                                       , 'dow_Mon':'dow_Mon'
                                                       , 'dow_Thu':'dow_Thu'
                                                       , 'dow_Tue':'dow_Tue'
                                                       , 'dow_Wed':'dow_Wed'})

writeCSV(kMeanDF, BASE_PATH + '\\data\\processed\\kmeans.csv')

## PCA

In [19]:
# PCA Analysis
PCAModel = PCA()
PCAModel.fit(scaleRegressorsDF)
stxp = PCAModel.transform(scaleRegressorsDF)

fig, ax = plt.subplots(figsize=(20,20))
ax = plt.scatter(stxp[:,0], stxp[:,1], c=kLabels, alpha=0.5)
plt.title('Biplot')
plt.axis('equal')
plt.xlabel('Principle Component 1')
plt.ylabel('Principle Component 2')
plt.legend(handles = ax.legend_elements(num=clusters_identified)[0], labels = ax.legend_elements(num=clusters_identified)[1], frameon=False)
plt.savefig(BASE_PATH + 'notebook\\main\\eda\\graphics\\pca_biplot.png')

In [20]:
PCAFeatures = range(PCAModel.n_components_)
            
fig, ax = plt.subplots(figsize=(20,20))
ax = plt.bar(PCAFeatures, PCAModel.explained_variance_)
plt.xticks(PCAFeatures)
plt.ylabel('Explained Variance')
plt.xlabel('PCA Components')
plt.title('Scree Plot of PCA Eigenvalues')
plt.savefig(BASE_PATH + 'notebook\\main\\eda\\graphics\\pca_explained_variance.png')

## Factor Analysis

In [21]:
# Factor Analysis
n_comps = 10 # Adjust based on +/- 1/2 eigenvalues

regressorList = regressors.columns.tolist()

methods = [
            ('PCA', PCA()),
            ('Unrotated Factor Analysis', FactorAnalysis(n_components=n_comps)),
            ('Varimax Factor Analysis', FactorAnalysis(n_components=n_comps, rotation='varimax')),
            ('Quartimax Factor Analysis', FactorAnalysis(n_components=n_comps, rotation='quartimax'))
          ]

fig, axes = plt.subplots(ncols=len(methods), figsize=(40,20))

for ax, (method, fa) in zip(axes, methods):
    fa.set_params(n_components=n_comps)
    fa.fit(scaleRegressorsDF)
    
    components = fa.components_.T
    print("\n\n %s :\n" % method)
    print(components)
    
    vmax = np.abs(components).max()
    ax.imshow(components, cmap='RdBu_r', vmax=vmax, vmin=-vmax)
    ax.set_yticks(np.arange(len(regressorList)))
    if ax.is_first_col():
         ax.set_yticklabels(regressorList)
    else:
        ax.set_yticklabels([])
    ax.set_title(str(method))
    ax.set_xticks([*range(n_comps)])
    ax.set_xticklabels([f'Comp. {i +1}' for i in range(n_comps)])

plt.savefig(BASE_PATH + 'notebook\\main\\eda\\graphics\\factor_analysis.png')



 PCA :

[[ 5.23851039e-01  1.58483190e-01  1.32844014e-01  8.38677341e-02
   1.14901593e-01  1.49748709e-01  1.76045722e-01  9.72964396e-02
   1.29279058e-02  1.70132991e-02]
 [ 5.16660898e-01  2.37448032e-01 -7.03128755e-02  1.54409922e-01
   9.53910324e-02 -1.72417027e-01 -9.86580323e-02  8.34504716e-03
  -4.31872999e-04  3.77710404e-02]
 [-5.85770843e-02  4.95392637e-01 -4.21393465e-02  2.16440840e-01
  -3.36479789e-01 -8.50383292e-03 -1.01249564e-02 -2.37606174e-02
   2.89952220e-04 -8.99891994e-02]
 [-2.89551339e-02  2.20655522e-01 -4.71292198e-02 -2.62299816e-02
   1.78825119e-02  6.88715178e-01 -3.66662466e-01 -3.60668820e-01
  -5.44396451e-02 -1.37458473e-01]
 [-4.29593658e-01  1.77849887e-01 -1.18258003e-01 -2.10042474e-02
  -1.22968472e-01 -2.91979434e-01  3.73930226e-01 -1.36044669e-01
  -5.32347226e-02 -1.88536620e-01]
 [ 6.13914181e-02 -1.27060224e-01 -1.63014913e-01 -1.98493751e-01
  -3.58201829e-01  9.43706354e-02 -1.79460101e-01  6.81435451e-01
   2.79214805e-02  1.15

## Write Prepped Dataset

In [22]:
writeCSV(
    scaleCombinedDF[['price'
                     , 'para1'
                     , 'loc1_0'
                     , 'loc1_1'
                     , 'loc1_2'
                     , 'loc1_3'
                     , 'loc1_6'
                     , 'loc1_7'
                     , 'dow_Mon'
                     , 'dow_Thu'
                     , 'dow_Tue']]
    , BASE_PATH + '\\data\\processed\\challenge1_processed_scaled.csv')

In [23]:
writeCSV(
    combinedDFSampled[['price'
                     , 'para1'
                     , 'loc1_0'
                     , 'loc1_1'
                     , 'loc1_2'
                     , 'loc1_3'
                     , 'loc1_6'
                     , 'loc1_7'
                     , 'dow_Mon'
                     , 'dow_Thu'
                     , 'dow_Tue']]
    , BASE_PATH + '\\data\\processed\\challenge1_processed.csv')

## GAM

In [ ]:
combinedDF = combinedDF[['price', 'para1', 'para2', 'para3', 'para4', 'loc1_0', 'loc1_1',
                         'loc1_2', 'loc1_3', 'loc1_4', 'loc1_5', 'loc1_6', 'loc1_7', 'loc1_8',
                         'loc1_9', 'dow_Fri', 'dow_Mon', 'dow_Thu', 'dow_Tue', 'dow_Wed']]

In [ ]:
#%%R -i combinedDF -w 5 -h 5 --units in -r 200
#
#install.packages(c('gam', 'glmnet', 'leaps'), repos='http://cran.us.r-project.org', quiet=TRUE)
#
#library(gam)
#library(glmnet)
#library(leaps)
#
#r_combinedDF <- combinedDF
#colnames(r_combinedDF)[1] <- make.names(colnames(combinedDF)[1])
#target <- colnames(combinedDF)[0]
#r_combinedDF <- na.omit(r_combinedDF)
#
#exp <- paste('gam( ', target, '~', sep='', collapse=NULL)
#
## Set degrees on polynomial
#dfr <- 3
#
#for (name in names(r_combinedDF[-c(1,1)]))
#{
#    exp <- paste(exp, 's(', name, ',', dfr')')
#    if(name != names(r_combinedDF)[length(r_combinedDF)]) {exp <- paste(exp, ' + ')}
#}
#
#exp <- paste(exp, ', data=r_combinedDF')
#gamFit <- eval(parse(text=exp))
#
#par(mfrow=c(3,3))
#plot(gamFit, se=TRUE, col='blue')
#summary(gamFit)
#
#gamPred <- predict.gam(gamFit)